In [ ]:
#It's forked from https://www.kaggle.com/code/tchaye59/efficientnet-tensorflow-baseline-tpu
#I have not yet been able to tune the parameters sufficiently, but I confirmed roughly 0.6 or better score with training 30 epochs!

In [ ]:
!nvidia-smi

In [ ]:
! pip uninstall -y tensorflow_datasets
! pip install tensorflow_datasets==4.4.0
!pip install -U keras-efficientnet-v2
!pip install tensorflow_addons
!pip install -U scikit-learn

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.preprocessing import LabelEncoder
import dill
import tensorflow as tf
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import backend as K 
import seaborn as sns
import random
import gc
from tqdm.notebook import tqdm
import tensorflow_addons as tfa
import sys
sys.path.append("../input/sorghum100cultivarjpgtfrecords512x512")
from fgvc_dataset import FGVCDataset
import keras_efficientnet_v2

In [ ]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code
tpu = None
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

    
#strategy,tpu = tf.distribute.MirroredStrategy(devices=["TPU:0", "TPU:1","TPU:2"]),True

print("Number of accelerators: ", strategy.num_replicas_in_sync)


AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

# We prepared the dataset in the following notebooks:

https://www.kaggle.com/tchaye59/images-to-jpeg-512x512

https://www.kaggle.com/tchaye59/512x512-images-to-tfrecords

In [ ]:
train_df = pd.read_csv("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv")
#remove 3329	.DS_Store	NaN
train_df = train_df[train_df.cultivar.notnull()]

test_df = pd.read_csv("../input/sorghum-id-fgvc-9/sample_submission.csv")
GCS_PATH = KaggleDatasets().get_gcs_path('sorghum100cultivarjpgtfrecords512x512')

In [ ]:
USE_SAVED_MODEL = False

# Load dataset

In [ ]:
%%time
data_dir= GCS_PATH+"/fgvc_dataset"
builder = FGVCDataset(data_dir=data_dir)
builder.download_and_prepare()
train_ds = builder.as_dataset()['train']
test_ds = builder.as_dataset()['test']

In [ ]:
def show_image_batch(images: list):
    """
    Displays a batch of image present in images
    """
    fig = plt.figure(figsize=(10,5))
    for idx in range(6):
        ax = plt.subplot(2, 3, idx+1)
        plt.imshow(images[idx])
        plt.axis("off")

def show_dataset(dataset):
    batch = next(iter(dataset))
    images, labels = batch
    
    plt.figure(figsize=(10, 10))
    for idx in range(9):
        ax = plt.subplot(3, 3, idx + 1)
        plt.imshow(images[idx].numpy().astype("uint8"))
        plt.title("Class: {}".format(labels[idx].numpy().decode()))
        plt.axis("off")

In [ ]:
show_dataset(train_ds.map(lambda data:(data['img'],data['cultivar'])).batch(9))

In [ ]:
le = dill.load(open("../input/sorghum100cultivarjpgtfrecords512x512/le.dill","rb"))
train_df["target"] = le.transform(train_df.cultivar)

In [ ]:
SEED = 123
HEIGHT = 512
WIDTH = 512
MAGNITUDE = 10
N_CLASSES = len(train_df.cultivar.unique())
BATCH_SIZE = 64 if tpu else 4
IMG_DIR = "../input/sorghum-id-fgvc-9" if not tpu else GCS_PATH

# TF functions

In [ ]:
# def data_augmentation():
#     return keras.Sequential([
#         #keras.layers.experimental.preprocessing.RandomZoom(0.2,seed=SEED),
#         #keras.layers.experimental.preprocessing.RandomCrop(HEIGHT//2, WIDTH//2,seed=SEED),
#         keras.layers.experimental.preprocessing.RandomContrast(0.1,seed=SEED),
#         keras.layers.experimental.preprocessing.RandomFlip(),
#         keras.layers.experimental.preprocessing.RandomRotation(0.2,seed=SEED),
#         #keras.layers.experimental.preprocessing.Resizing(HEIGHT, WIDTH),
#     ])
# daug = data_augmentation()

# use RandAugment containing in keras_efficientnet_v2(origin source is https://github.com/tensorflow/models/blob/master/official/vision/ops/augment.py)
class RandomProcessImage:
    def __init__(self, target_shape=(300, 300), magnitude=0, keep_shape=False):
        self.target_shape, self.magnitude, self.keep_shape = target_shape, magnitude, keep_shape
        self.target_shape = target_shape if len(target_shape) == 2 else target_shape[:2]
        if magnitude > 0:
            from keras_efficientnet_v2 import augment

            translate_const, cutout_const = 100, 40
#             translate_const = int(target_shape[0] * 10 / magnitude)
            cutout_const = int(target_shape[0] * 40 / 224)
            print(">>>> RandAugment: magnitude = %d, translate_const = %d, cutout_const = %d" % (magnitude, translate_const, cutout_const))
            aa = augment.RandAugment(num_layers = 2, magnitude=magnitude, translate_const=translate_const, cutout_const=cutout_const)
            # aa.available_ops = ["AutoContrast", "Equalize", "Invert", "Rotate", "Posterize", "Solarize", "Color", "Contrast", "Brightness", "Sharpness", "ShearX", "ShearY", "TranslateX", "TranslateY", "Cutout", "SolarizeAdd"]
            # aa.available_ops = ["AutoContrast", "Equalize", "Invert", "Rotate", "Posterize", "Solarize", "Color", "Contrast", "Brightness", "Sharpness", "TranslateX", "TranslateY", "SolarizeAdd"]
            self.process = lambda img: aa.distort(img)
        elif magnitude == 0:
            self.process = lambda img: tf.image.random_flip_left_right(img)
        else:
            self.process = lambda img: img

    def __call__(self, datapoint):
        image = datapoint
        if self.keep_shape:
            cropped_shape = tf.reduce_min(tf.keras.backend.shape(image)[:2])
            image = tf.image.random_crop(image, (cropped_shape, cropped_shape, 3))

        input_image = tf.image.resize(image, self.target_shape)
        input_image = self.process(input_image)
        # input_image = (tf.cast(input_image, tf.float32) - 127.5) / 128
        return input_image

daug = RandomProcessImage((WIDTH, HEIGHT), MAGNITUDE, keep_shape=True)

In [ ]:
def load_train_image(data):
    img = data['img']
    cultivar = data['cultivar']
    name = data['name']
    target = data['target']
    # Resize image
    img = tf.image.resize(img,(WIDTH, HEIGHT),)
    return img,target

def load_test_image(data):
    img = data['img']
    name = data['name']
    # Resize image
    img = tf.image.resize(img,(WIDTH, HEIGHT),)
#     img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img,name

def prepare_train_dataset(train_ds):
    steps = len(train_ds)//BATCH_SIZE
    train_ds = train_ds.repeat().shuffle(5000).map(load_train_image,num_parallel_calls=AUTO)
    train_ds = train_ds.map(lambda x,y:(daug(x),y),num_parallel_calls=AUTO).batch(BATCH_SIZE).prefetch(100)
    return steps,train_ds

def prepare_submission_dataset(ds):
    ds = ds.map(load_test_image,num_parallel_calls=AUTO)
    ds = ds.batch(BATCH_SIZE*2).prefetch(100)
    return ds

# Show augmented images

In [ ]:
show_dataset(train_ds.map(lambda data:(data['img'],data['cultivar'])).map(lambda x,y:(daug(x),y)).batch(9))

# Model 

In [ ]:
with strategy.scope():
    #Exclude model top layers by set num_classes=0
    basemodel = keras_efficientnet_v2.EfficientNetV2M(input_shape=(WIDTH, HEIGHT, 3), drop_connect_rate=0.2, num_classes=0, include_preprocessing=True, pretrained="imagenet")
    basemodel.trainable = True

    image_input = tf.keras.layers.Input(shape=(WIDTH,HEIGHT,3))
    out = basemodel(image_input)
    out = tf.keras.layers.GlobalAveragePooling2D()(out)
    out = tf.keras.layers.Dropout(0.2)(out)
    out = tf.keras.layers.Dense(100, activation="softmax")(out)

    model = tf.keras.Model(image_input, out)

    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss='sparse_categorical_crossentropy',
                  metrics=["accuracy"])

# Train

In [ ]:
with strategy.scope():
    if not USE_SAVED_MODEL:
        steps_per_epoch,ds =  prepare_train_dataset(train_ds)
        callback = tf.keras.callbacks.EarlyStopping(monitor='acc',mode='max', patience=20)
        ckp_callback = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath=f'model.h5',
                                                    save_weights_only=True,
                                                    monitor='acc',
                                                    mode='max',
                                                    options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'),
                                                    save_best_only=True)
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='acc',mode='max',factor=0.2,patience=3, min_lr=1e-5)
        callbacks=[callback,ckp_callback,reduce_lr]
        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                      loss=tf.keras.losses.sparse_categorical_crossentropy,
                      metrics=['acc',])

        history = model.fit(ds,
                            steps_per_epoch=steps_per_epoch,
                            epochs=30,
                            callbacks=callbacks)

# Submit

In [ ]:
if not USE_SAVED_MODEL:
    model.load_weights('model.h5')
else:
    model.load_weights('../input/sorghum100efficientnetbaselinemodel/model.h5')

In [ ]:
def predict(x):
    return model(x,training=False)
@tf.function
def dist_predict(dist_inputs):
    res = strategy.run(predict, args=(dist_inputs,))
    return res

In [ ]:
%%time
with strategy.scope():
    ds = prepare_submission_dataset(test_ds)
    dist_ds = strategy.experimental_distribute_dataset(ds)
    
    all_names = []
    all_targets = []
    for img,names in tqdm(dist_ds):
        preds = dist_predict(img)
        if tpu:
            preds = tf.concat(preds.values,axis=0)
            names = tf.concat(names.values,axis=0)
        preds = preds.numpy()
        names = names.numpy()
        preds = np.argmax(preds,axis=-1)
        all_targets.extend(list(preds))
        all_names.extend([s.decode('ascii') for s in names])

In [ ]:
sub_df = pd.DataFrame({
    "filename":all_names,
    "cultivar":le.inverse_transform(all_targets)
})
sub_df.head(5)

In [ ]:
sub_df.to_csv("submission.csv",index=False)